In [ ]:
# Imports
import babypandas as bpd
import numpy as np

import matplotlib.pyplot as plt
import warnings; warnings.simplefilter('ignore')

plt.style.use('fivethirtyeight')

# Animations
from ipywidgets import interact

# Lecture 22 – The Normal Curve

## DSC 10, Fall 2021

### Announcements

- Homework 6 is due **Tuesday 11/16 at 11:59pm**.
- Lab 7 is due **Sunday 11/21 at 11:59pm**.
- The Final Project is due **Wednesday 12/1 at 11:59pm**.
    - Start early.
    - Come to office hours.
    - Work with a partner from your team – use [this sheet](https://docs.google.com/spreadsheets/d/19tnetCMN1z9Kl57kKLtqNzT5ppjub1nDUniNjA-iXKE/edit#gid=0) to find one. If you're having trouble finding a partner, let the instructors know.
- Consider taking Suraj's [History of Data Science](https://dsc-courses.github.io/dsc90-2022-wi/) seminar next quarter!

### Agenda

- The normal curve and its many properties.
- Standard units and normal approximation.
- Revisit center and spread in the context of the normal curve.
- Introduction to the Central Limit Theorem.

## The normal curve

### The data

We'll work with a dataset containing the heights and weights of 5000 adult males.

In [ ]:
height_and_weight = bpd.read_csv('data/height_and_weight.csv')
height_and_weight

### Distribution of heights

In [ ]:
height_and_weight.plot(kind='hist', y='Height', density=True, ec='w', bins=20, alpha=0.8);

### Distribution of weights

In [ ]:
height_and_weight.plot(kind='hist', y='Weight', density=True, ec='w', bins=20, alpha=0.8, color='C1');

### A familiar shape

- We've seen this bell-like shape before.
- These bells are different in two key aspects: center and spread.

In [ ]:
# Overlaid histogram
height_and_weight.plot(kind='hist', density=True, bins=120, alpha=0.8);

Let's see if we can **transform** these distributions so that they look the same. 👀

### Centering

To **center** a distribution, subtract the mean from each value.

In [ ]:
mean_height = height_and_weight.get('Height').mean()
mean_height

In [ ]:
centered_height = height_and_weight.get('Height') - mean_height
centered_height

In [ ]:
mean_weight = height_and_weight.get('Weight').mean()
mean_weight

In [ ]:
centered_weight = height_and_weight.get('Weight') - mean_weight
centered_weight

In [ ]:
centered_height_and_weight = height_and_weight.assign(
    Height=centered_height,
    Weight=centered_weight
)
centered_height_and_weight

### Centering

Let's plot these distributions now that they're centered.

In [ ]:
centered_height_and_weight.plot(kind='hist', density=True, bins=120, alpha=0.8);

Note that the mean of any centered distribution is 0.

In [ ]:
centered_height_and_weight.get('Height').mean()

In [ ]:
centered_height_and_weight.get('Weight').mean()

### Scaling

- Remember, our current goal is to transform the two distributions so that they look very similar.
- Right now they are both centered, but have different widths.
- To give them the same width, we **divide by the standard deviation**.
- Data that is centered and scaled is **standardized**.

In [ ]:
std_height = np.std(height_and_weight.get('Height'))
std_height

In [ ]:
std_weight = np.std(height_and_weight.get('Weight'))
std_weight

In [ ]:
standardized_height_and_weight = centered_height_and_weight.assign(
    Height=centered_height_and_weight.get('Height') / std_height,
    Weight=centered_height_and_weight.get('Weight') / std_weight
)

Note that the mean of any standardized distribution is 0, and the standard deviation of any standardized distribution is 1.

In [ ]:
np.std(standardized_height_and_weight.get('Height'))

In [ ]:
np.std(standardized_height_and_weight.get('Weight'))

### Standardized histograms

Now that we've both centered and scaled the two distributions, let's see how they look.

In [ ]:
standardized_height_and_weight.plot(kind='hist', density=True, bins=120, alpha=0.8);

These both look pretty similar!

### The standard normal curve

- The distributions we've seen look essentially the same once standardized.
- This shape is called the **standard normal curve**.

$$
\phi(z) = \frac{1}{\sqrt{2 \pi}} e^{-\frac{1}{2}z^2}
$$


- You don't need to know the formula – just the shape!

### The standard normal curve

In [ ]:
def normal_curve(z):
    return 1 / np.sqrt(2*np.pi) * np.exp((-z**2)/2)

In [ ]:
x = np.linspace(-4, 4, 1000)
y = normal_curve(x)

plt.plot(x, y, color='black');
plt.xlabel('$z$');
plt.title(r'$\phi(z) = \frac{1}{\sqrt{2 \pi}} e^{-\frac{1}{2}z^2}$');

### Heights/weights are approximately normal

- If a distribution follows this shape, we say it is roughly "normal".

In [ ]:
standardized_height_and_weight.plot(kind='hist', density=True, bins=120, alpha=0.8);
plt.plot(x, y, color='black', linestyle='--', label='Normal')
plt.legend(loc='upper right');

### The standard normal distribution

- Think of the normal curve as a "continuous histogram".
- Its mean and median are both 0 – it is symmetric.
- It has inflection points at $\pm 1$.
    - More on this later.
- Like a histogram:
    - The total area is 1.
    - The area between $a$ and $b$ is proportion of values between $a$ and $b$.

In [ ]:
def normal_area(a, b, bars=False):
    ix = (x >= a) & (x <= b)
    plt.plot(x, y, color='black')
    plt.fill_between(x[ix], y[ix], color='gold')
    if bars:
        plt.axvline(a, color='red')
        plt.axvline(b, color='red')
    plt.title(f'Area between {np.round(a, 2)} and {np.round(b, 2)}')

In [ ]:
interact(normal_area, a=(-4, 3, 0.25), b=(-4, 4, 0.25), bars=False);

### The standard normal "CDF"

- Areas under curves are typically found via integration.
- However, there is no closed-form indefinite integral for the normal curve.
- Instead, we approximate through computation.
- The function `scipy.stats.norm.cdf(z)` computes the area under the standard normal curve to the left of `z`. In other words,

$$
\texttt{scipy.stats.norm.cdf(z)} = \int_{- \infty}^z \phi(x) \, dx
$$
- cdf stands for "cumulative density function".

### Discussion Question

What will `scipy.stats.norm.cdf(0)` evaluate to?

- A) $\pi$
- B) $0$
- C) $1$
- D) $\frac{1}{2}$
- E) $42$

### To answer, go to [menti.com](https://menti.com) and enter the code 3467 8926.

### Answer

In [ ]:
from scipy import stats
stats.norm.cdf(0)

In [ ]:
normal_area(-np.inf, 0)

### Example: area to the right of 2

The area to the left of 2 can be computed using `stats.norm.cdf(2)`.

In [ ]:
normal_area(-np.inf, 2)

In [ ]:
stats.norm.cdf(2)

Then, the area to the right of 2 is 1 minus the area to the left of 2.

In [ ]:
normal_area(2, np.inf)

In [ ]:
1 - stats.norm.cdf(2)

### Example: area between [-1, 0]

How would we compute the area between -1 and 0?

In [ ]:
normal_area(-1, 0)

Strategy:

$$(\text{area from $-1$ to $0$}) = (\text{area from $-\infty$ to $0$}) - (\text{area from -$\infty$ to $-1$})$$

In words: take the area to the left of 0, and subtract the area to the left of 1.

In [ ]:
plt.fill_between(x[(x >= -np.inf) & (x <= -1)], y[(x >= -np.inf) & (x <= -1)], color='blue')
normal_area(-1, 0)

In [ ]:
stats.norm.cdf(0) - stats.norm.cdf(-1)

### General strategy for finding area

- The area between $[a, b]$ is 

```py
stats.norm.cdf(b) - stats.norm.cdf(a)
```

## Standard units and normal approximation

### Standard units

**Question:** What proportion of weights are between 200 and 225?

- We can determine the answer exactly, because we have access to all of the data.
- But, let's use fact that data is approximately normal.
    - This is done often in settings where you don't have all of the data, just the mean and standard deviation.

### Histogram area within [200, 225]

In [ ]:
height_and_weight.plot(kind='hist', y='Weight', density=True, ec='w', bins=20, alpha=0.8, color='C1');
plt.axvline(200, color='black')
plt.axvline(225, color='black');

### Normal curve area within [?, ?]...

The question becomes, what two endpoints should we look for the area between on the standard normal curve?

In [ ]:
plt.plot(x, y, color='black');

### Standard units

- Define $$z(x) = \frac{x - \text{mean of all $x$s}}{\text{SD of all $x$s}}$$
- $z(x)$ converts $x$ to **standard units**, which represents how many SDs above average $x$ is.
- Converting a variable to standard units is the same as **standardizing** it, which is what we did earlier in the lecture.

### Standard units and area

- **The $x$-axis in a plot of the standard normal curve is in standard units.**
    - e.g. the area between -1 and 1 is the proportion of values within 1 standard deviation of the mean.
- Suppose a distribution is (roughly) normal. Then, these are two are approximately equal:
    - The area between $a$ and $b$ in histogram.
    - The area between $z(a)$ and $z(b)$ in a standard normal curve.

In [ ]:
plt.plot(x, y, color='black');

### Standard units

What does a weight of 225 become after standardization?

$$z(225) = \frac{225 - \text{mean weight}}{\text{SD of weight}}$$

In [ ]:
225 - mean_weight

In [ ]:
(225 - mean_weight) / std_weight

- Interpretation: 225 is 1.92 standard deviations above the mean weight.
- 225 becomes 1.92 in **standard units**.

### Standard units and area

In [ ]:
left = (200 - mean_weight) / std_weight
left

In [ ]:
right = (225 - mean_weight) / std_weight
right

In [ ]:
# Area between left and right in the standard normal curve
approximation = stats.norm.cdf(right) - stats.norm.cdf(left)
approximation

**Interpretation:** approximately 22.8% of weights are between 200 and 225 (inclusive).

**Note that we only needed the mean and standard deviation of the weights to calculate this!**

### Normal curve area between $z(200)$ and $z(225)$

In [ ]:
normal_area(left, right, bars=True)
plt.axvline(.656, color='C1')
plt.axvline(1.92, color='C1')
plt.text(.656, .45, '$z(200)$', ha='center')
plt.text(1.92, .45, '$z(225)$', ha='center')
plt.title('');

### Checking the approximation

In [ ]:
# True proportion of values between 200 and 225
height_and_weight[
    (height_and_weight.get('Weight') >= 200) &
    (height_and_weight.get('Weight') <= 225)
].shape[0] / height_and_weight.shape[0]

In [ ]:
# Approximation using the standard normal curve
approximation

Pretty good for an approximation! 🤩

### Summary: normal approximations

- Suppose we know the following about a distribution:
    - it is approximately normal,
    - its mean and standard deviation,
- Then, we can use the standard normal curve to compute the proportion of values in a given range without needing all of the data.

## Center and spread, revisited

### Special cases

- As we just discovered, the $x$-axis in the standard normal curve represents standard units.
- Often times, we want to know the area within $z$ standard deviations of the mean.

|Percent in Range | Normal Distributions|
|---|---|
|$\text{average} \pm 1 \ \text{SD}$ | $\approx 68\%$ |
|$\text{average} \pm 2 \text{SDs}$ | $\approx 95\%$ |
|$\text{average} \pm 3\text{SDs}$ | $\approx 99.73\%$ |

### 68% of values are in [-1, 1]

In [ ]:
normal_area(-1, 1, bars=True)

In [ ]:
stats.norm.cdf(1) - stats.norm.cdf(-1)

This means that if a variable follows the standard normal distribution, 68% of values will be within 1 SD of the mean.

### 95% of values are in [-2, 2]

In [ ]:
normal_area(-2, 2, bars=True)

In [ ]:
stats.norm.cdf(2) - stats.norm.cdf(-2)

This means that if a variable follows the standard normal distribution, 95% of values will be within 2 SDs of the mean.

- This also means that 5% of values will be outside of 2 SDs from the mean.
- Since the normal curve is symmetric, it also means that 2.5% of values will be more than 2 SDs above the mean, and 2.5% of values will be more than 2 SDs below the mean.

### Chebyshev's inequality and the normal curve

- Last class, we looked at Chebyshev's inequality, which stated that the proportion of data within $z$ SDs of the mean is **at least** $1-1/z^2$.
    - This works for any distribution, and is a lower bound.
- If we know that the distribution is normal, we can do even better:


|Percent in Range | All Distributions (via Chebyshev's Inequality) | Normal Distributions|
|---|---|---|
|$\text{average} \pm 1 \ \text{SD}$ | $\geq 0\%$ | $\approx 68\%$ |
|$\text{average} \pm 2\text{SDs}$ | $\geq 75\%$ | $\approx 95\%$ |
|$\text{average} \pm 3\text{SDs}$ | $\geq 88\%$ | $\approx 99.73\%$ |

- The percentages you see for normal distributions above are approximate, but are not lower bounds.

### Points of inflection

- Earlier, we mentioned that the standard normal curve has points of inflection at $z = \pm 1$.
    - Inflection point: where the curve goes from "opening down" to "opening up."
- We know that the $x$-axis of the standard normal curve represents standard units, so this means the points of inflection are at 1 standard deviation above and below the mean.
- This means that if a distribution is roughly normal, we can determine its standard deviation by looking for how far its point of inflection is from its center.

In [ ]:
height_and_weight.plot(kind='hist', y='Height', density=True, ec='w', bins=40, alpha=0.8);
plt.xticks(np.arange(60, 78, 2));

- The center appears to be around 69.
- The points of inflection appear to be around 66 and 72.
- So, the standard deviation is roughly 72 - 69 = 3.

In [ ]:
np.std(height_and_weight.get('Height'))

## Central Limit Theorem

### Something to think about...

When looking at the empirical distribution of a statistic, we've often seen a bell curve. Why is that? 🤔

### Bell-shaped data

- In real life, data is sometimes bell-shaped.
- We've seen this to be true for height and weight.
- There are many more examples...

### Example: Baseball runners left on base ⚾️

In [ ]:
baseball = bpd.read_csv('data/baseball.csv').get(['left_on_base'])
baseball.plot(kind='hist', y='left_on_base', bins=20, density=True, ec='w');

### Example: Natural sleep duration 😴

In [ ]:
sleep = bpd.read_csv('data/sleep.csv').get(['Sleep Duration'])
sleep.plot(kind='hist', y='Sleep Duration', bins=30, density=True, ec='w');

### But...

- **...most data isn't bell-shaped!**
- Bell-shaped distributions are symmetric and only have one peak.
- But in reality, distributions are often skewed to one side and have multiple peaks.

### Example: City of San Diego employee salaries 💰

In [ ]:
salaries = bpd.read_csv('data/2020_salaries.csv').get(['TotalWages'])
salaries.plot(kind='hist', y='TotalWages', bins=30, density=True, ec='w');

### Example: Flight delays ✈️

In [ ]:
flights = bpd.read_csv('data/united_summer2015.csv').get(['Delay'])
flights.plot(kind='hist', y='Delay', bins=np.arange(-25, 200, 5), density=True, ec='w');

### But...

- It turns out that **even when the population distribution isn't bell-shaped, the empirical distribution of sample statistics often are**.
- Let's consider the distribution of the sample mean.

### Example: Flight delays ✈️

- We want to know the average flight delay.
- To do this, we take a random sample of flights and compute the sample mean.
- The sample mean could have been different, so we'd like some sense of how much the sample mean could vary – we'd like to know the distribution of the sample mean.
- We now know that we can use the bootstrap to estimate this distribution through just a single sample.
    - But in this case, since we have the population, we'll just repeatedly sample from it.

In [ ]:
n_samples = 1000

sample_means = np.array([])

for i in np.arange(n_samples):
    sample = flights.sample(2000, replace=True) # Sampling directly from the population, with replacement
    sample_mean = sample.get('Delay').mean()
    sample_means = np.append(sample_means, sample_mean)
    
# Look at just the first 10 sample means
sample_means[:10]

In [ ]:
bpd.DataFrame().assign(SampleMeans=sample_means).plot(kind='hist', bins=20, density=True, ec='w')
plt.xlabel('Mean Delay')
plt.title('Distribution of the Sample Mean');

### What just happened ⁉️

- The population was not bell-shaped – it was extremely skewed.
- But distribution of sample mean **is** roughly bell-shaped.
- Most of the bootstrap distributions we've seen in this class were bell-shaped.
- This appears to be a deep fact about randomness.

### Central Limit Theorem

> The Central Limit Theorem (CLT) says that the probability distribution of the **sum or average** of a large random sample drawn with replacement will be roughly normal, regardless of the distribution of the population from which the sample is drawn.

More on this next time.

## Summary

### Summary, next time

- The standard normal curve is a symmetric, bell-shaped distribution.
- If a distribution is roughly normal, we know that roughly 68% of the data lies within 1 SD of the mean, 95% of the data lies within 2 SDs of the mean, and 99.73% of the data lies within 3 SDs of the mean.
    - These are more specific percentages than Chebyshev's inequality gave us; Chebyshev's inequality is more general, as it applies to all distributions.
- Many distributions in practice resemble the standard normal curve.
    - To convert a variable to standard units (i.e. to standardize a variable), we subtract each value by its mean and divide by its standard deviation.
    - If a variable is standardized and roughly bell-shaped, we can use the normal curve to approximate the proportion of values in an interval.
    - The $x$-axis of the normal curve is in standard units.
- The Central Limit Theorem says that distributions of sums and averages of large random samples will be bell-shaped.
- **Next time:** more on CLT.